# Google Sheets Legacy Data Pipeline for Banking Churn POC

This notebook handles the Google Sheets legacy data pipeline for the Apex National Bank churn prediction POC.

## Overview

Google Sheets represents our **Legacy System** - spreadsheets maintained by branch staff:
- **Branches** - Branch master data (10 branches)
- **Customer Notes** - Manual notes from relationship managers (churn signals!)

## Churn Signal Logic

Manual notes from branch staff often contain early churn signals:

| Segment | Note Type | Example Notes |
|---------|-----------|---------------|
| Active | Positive | "Great customer, interested in mortgage" |
| At-Risk | Warning | "Complained about fees, shopping competitors" |
| Churned | Negative | "Requested closure", "Moving to competitor" |

## Entity Resolution

Customers are linked via **email**:

```
ERPNext (email_id) <---> Google Sheets (customer_email)
```

## Notebook Sections

1. **Configuration** - Environment setup and credentials
2. **Google Sheets Client** - API connection
3. **Data Ingestion** - Create branches and customer notes
4. **Data Extraction** - Export to CSV for Databricks

---
## 1. Configuration

### Prerequisites

1. Google Cloud project with Sheets API enabled
2. Service account credentials JSON file in `docs/`
3. Google Sheet created and shared with service account

### Install Dependencies

```bash
pip install gspread google-auth
```

In [1]:
# Install dependencies if not present
try:
    import gspread
    from google.oauth2.service_account import Credentials
    print("gspread and google-auth already installed")
except ImportError:
    print("Installing gspread and google-auth...")
    !pip install gspread google-auth
    import gspread
    from google.oauth2.service_account import Credentials
    print("Installation complete!")

gspread and google-auth already installed


In [2]:
# Required packages
import os
import re
import json
import csv
import random
import uuid
from datetime import datetime, timedelta
from typing import List, Dict, Any
from pathlib import Path
import pandas as pd

print("Packages imported successfully!")

Packages imported successfully!


In [3]:
# =============================================================================
# CONFIGURATION - Auto-load from .env file
# =============================================================================

def load_env_file(env_path: str) -> dict:
    """
    Load credentials from .env file (PowerShell format).
    """
    credentials = {}
    try:
        with open(env_path, 'r') as f:
            for line in f:
                match = re.match(r'\$env:(\w+)\s*=\s*["\']([^"\']*)["\']', line.strip())
                if match:
                    var_name, value = match.groups()
                    credentials[var_name] = value
        print(f"Loaded {len(credentials)} credentials from {env_path}")
    except FileNotFoundError:
        print(f"Warning: {env_path} not found, using environment variables")
    return credentials

# Load from .env file
ENV_FILE = Path("../../docs/.env")
DOCS_DIR = Path("../../docs")
env_creds = load_env_file(str(ENV_FILE))

# Google Sheets Configuration
GOOGLE_SHEETS_ID = env_creds.get("GOOGLE_SHEETS_ID", os.getenv("GOOGLE_SHEETS_ID", ""))
GOOGLE_CREDENTIALS_FILE = env_creds.get("GOOGLE_CREDENTIALS_FILE", os.getenv("GOOGLE_CREDENTIALS_FILE", ""))
GOOGLE_CREDENTIALS_PATH = DOCS_DIR / GOOGLE_CREDENTIALS_FILE

# Data paths
ERP_CUSTOMERS_PATH = Path("../../data/raw/erp_customers.csv")
OUTPUT_DIR = Path("../../data/raw")

# Date range (same as other systems)
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2026, 1, 11)

# Check configuration
creds_exist = GOOGLE_CREDENTIALS_PATH.exists()
sheet_id_set = bool(GOOGLE_SHEETS_ID)

print(f"\nConfiguration:")
print(f"  Spreadsheet ID: {GOOGLE_SHEETS_ID[:20]}..." if GOOGLE_SHEETS_ID else "  Spreadsheet ID: NOT SET")
print(f"  Credentials file: {GOOGLE_CREDENTIALS_PATH}")
print(f"  Credentials exist: {creds_exist}")

if not creds_exist:
    print("\nWARNING: Credentials file not found!")
if not sheet_id_set:
    print("\nWARNING: Spreadsheet ID not set in .env!")

Loaded 13 credentials from ..\..\docs\.env

Configuration:
  Spreadsheet ID: 1pcpF4IJqRv7aVLMmApU...
  Credentials file: ..\..\docs\banking-churn-poc-5c5c657a7df3.json
  Credentials exist: True


In [4]:
# =============================================================================
# REFERENCE DATA
# =============================================================================

# Branch data for Apex National Bank (10 branches)
BRANCHES = [
    {"branch_id": "BR001", "branch_name": "Downtown Main", "region": "Central", "manager_name": "Sarah Johnson", "staff_count": 25, "opened_date": "2015-03-15", "performance_rating": 4.5},
    {"branch_id": "BR002", "branch_name": "Westside Plaza", "region": "West", "manager_name": "Michael Chen", "staff_count": 18, "opened_date": "2017-06-20", "performance_rating": 4.2},
    {"branch_id": "BR003", "branch_name": "Northgate Mall", "region": "North", "manager_name": "Emily Rodriguez", "staff_count": 15, "opened_date": "2018-09-10", "performance_rating": 4.0},
    {"branch_id": "BR004", "branch_name": "Eastside Business", "region": "East", "manager_name": "David Kim", "staff_count": 20, "opened_date": "2016-01-25", "performance_rating": 4.3},
    {"branch_id": "BR005", "branch_name": "South Central", "region": "South", "manager_name": "Amanda Foster", "staff_count": 12, "opened_date": "2019-04-05", "performance_rating": 3.8},
    {"branch_id": "BR006", "branch_name": "University District", "region": "Central", "manager_name": "James Wilson", "staff_count": 14, "opened_date": "2020-02-14", "performance_rating": 4.1},
    {"branch_id": "BR007", "branch_name": "Tech Hub", "region": "West", "manager_name": "Lisa Park", "staff_count": 22, "opened_date": "2019-11-30", "performance_rating": 4.6},
    {"branch_id": "BR008", "branch_name": "Harbor View", "region": "South", "manager_name": "Robert Martinez", "staff_count": 16, "opened_date": "2018-07-22", "performance_rating": 3.9},
    {"branch_id": "BR009", "branch_name": "Airport Business", "region": "East", "manager_name": "Jennifer Lee", "staff_count": 10, "opened_date": "2021-05-18", "performance_rating": 4.0},
    {"branch_id": "BR010", "branch_name": "Suburban Heights", "region": "North", "manager_name": "Thomas Brown", "staff_count": 13, "opened_date": "2022-01-10", "performance_rating": 4.2},
]

# Note templates by segment (churn signal!)
NOTE_TEMPLATES = {
    "active": [
        ("positive", "Excellent customer - always maintains high balance"),
        ("positive", "Interested in mortgage products, follow up next quarter"),
        ("positive", "Referred 2 new customers this month"),
        ("positive", "Very satisfied with service, left positive feedback"),
        ("positive", "Upgraded to premium account, engaged customer"),
        ("positive", "Long-term customer, strong relationship"),
        ("neutral", "Routine account review completed"),
        ("neutral", "Updated contact information"),
    ],
    "at_risk": [
        ("warning", "Complained about monthly fees - consider waiver"),
        ("warning", "Mentioned competitor rates during call"),
        ("warning", "Reduced account activity noticed"),
        ("warning", "Asked about account closure process"),
        ("warning", "Unhappy with recent service experience"),
        ("warning", "Requested statement for external review"),
        ("warning", "Balance declining, may be moving funds"),
        ("neutral", "Follow-up call scheduled for retention"),
    ],
    "churned": [
        ("negative", "Customer requested account closure"),
        ("negative", "Moving to competitor - cited better rates"),
        ("negative", "Closed all accounts, dissatisfied with fees"),
        ("negative", "Relocating out of service area"),
        ("negative", "Filed formal complaint before leaving"),
        ("warning", "Final retention attempt unsuccessful"),
        ("negative", "Account closed per customer request"),
        ("negative", "Lost to digital-only bank"),
    ],
}

# Staff names for note authors
STAFF_NAMES = [
    "J. Smith", "M. Garcia", "K. Williams", "R. Johnson", "L. Brown",
    "S. Davis", "A. Miller", "C. Wilson", "E. Taylor", "P. Anderson",
]

print(f"Reference data loaded:")
print(f"  Branches: {len(BRANCHES)}")
print(f"  Note templates: {sum(len(v) for v in NOTE_TEMPLATES.values())}")

Reference data loaded:
  Branches: 10
  Note templates: 24


---
## 2. Google Sheets Client

Connect to Google Sheets using service account credentials.

In [5]:
def connect_google_sheets():
    """
    Connect to Google Sheets using service account credentials.
    """
    try:
        # Define scopes
        scopes = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]
        
        # Load credentials
        credentials = Credentials.from_service_account_file(
            str(GOOGLE_CREDENTIALS_PATH),
            scopes=scopes
        )
        
        # Create client
        client = gspread.authorize(credentials)
        
        # Open spreadsheet
        spreadsheet = client.open_by_key(GOOGLE_SHEETS_ID)
        
        print(f"Connected to Google Sheets!")
        print(f"  Spreadsheet: {spreadsheet.title}")
        print(f"  Sheets: {[ws.title for ws in spreadsheet.worksheets()]}")
        
        return spreadsheet
    except Exception as e:
        print(f"Connection error: {e}")
        return None


# Connect
if creds_exist and sheet_id_set:
    spreadsheet = connect_google_sheets()
else:
    print("Skipping connection - credentials not properly configured")
    spreadsheet = None

Connected to Google Sheets!
  Spreadsheet: Apex Bank - Branch Data
  Sheets: ['branches', 'customer_notes ']


In [6]:
# Load ERPNext customers to link notes
print("\n" + "=" * 60)
print("ERPNext Customers (to link)")
print("=" * 60)

if ERP_CUSTOMERS_PATH.exists():
    erp_customers = pd.read_csv(ERP_CUSTOMERS_PATH)
    
    print(f"\nLoaded {len(erp_customers)} customers from ERPNext")
    print(f"\nSegment distribution:")
    print(erp_customers['website'].value_counts())  # 'website' stores segment
    
    print(f"\nSample customers:")
    print(erp_customers[['customer_name', 'email_id', 'website']].head())
else:
    print(f"ERPNext customer file not found: {ERP_CUSTOMERS_PATH}")
    erp_customers = None


ERPNext Customers (to link)

Loaded 502 customers from ERPNext

Segment distribution:
website
active     311
at_risk    126
churned     63
Name: count, dtype: int64

Sample customers:
     customer_name                 email_id  website
0  Adnan Ahmed 831  adnan.ahmed31@gmail.com   active
1  Adnan Begum 248  adnan.begum45@gmail.com   active
2  Adnan Begum 595  adnan.begum26@gmail.com   active
3  Adnan Malik 605  adnan.malik70@gmail.com  at_risk
4  Adnan Malik 886  adnan.malik79@gmail.com   active


---
## 3. Data Ingestion

Create branch data and customer notes in Google Sheets.

### Churn Signal Implementation

| Segment | Notes Count | Note Types |
|---------|-------------|------------|
| Active | 1-3 | Mostly positive |
| At-Risk | 2-4 | Warning notes |
| Churned | 2-5 | Negative/closure notes |

In [7]:
def create_branches_sheet(spreadsheet) -> bool:
    """
    Create or update the branches sheet with branch master data.
    """
    print("\n" + "=" * 60)
    print("Creating Branches Sheet")
    print("=" * 60)
    
    try:
        # Get or create branches worksheet
        try:
            ws = spreadsheet.worksheet("branches")
            ws.clear()  # Clear existing data
            print("  Found existing 'branches' sheet, clearing...")
        except gspread.exceptions.WorksheetNotFound:
            ws = spreadsheet.add_worksheet(title="branches", rows=20, cols=10)
            print("  Created new 'branches' sheet")
        
        # Prepare data
        headers = ["branch_id", "branch_name", "region", "manager_name", "staff_count", "opened_date", "performance_rating"]
        rows = [headers]
        
        for branch in BRANCHES:
            row = [branch[col] for col in headers]
            rows.append(row)
        
        # Write to sheet
        ws.update(range_name='A1', values=rows)
        
        print(f"  Wrote {len(BRANCHES)} branches to sheet")
        return True
        
    except Exception as e:
        print(f"  Error: {e}")
        return False

In [8]:
def determine_notes_count(segment: str) -> int:
    """
    Determine number of notes based on customer segment.
    At-risk and churned customers tend to have more notes.
    """
    if segment == "churned":
        return random.randint(2, 5)  # More notes documenting issues
    elif segment == "at_risk":
        return random.randint(2, 4)  # Warning notes
    else:  # active
        return random.randint(0, 3)  # Occasional positive notes


def get_note_dates(segment: str, start: datetime, end: datetime) -> List[datetime]:
    """
    Generate note dates based on segment behavior.
    """
    num_notes = determine_notes_count(segment)
    dates = []
    
    if segment == "churned":
        # Notes cluster near churn date
        churn_date = end - timedelta(days=random.randint(30, 180))
        for _ in range(num_notes):
            offset = random.randint(-90, 30)  # Notes around churn
            note_date = churn_date + timedelta(days=offset)
            if start <= note_date <= end:
                dates.append(note_date)
    else:
        # Random dates throughout period
        delta = (end - start).days
        for _ in range(num_notes):
            random_days = random.randint(0, delta)
            dates.append(start + timedelta(days=random_days))
    
    return sorted(dates)


def get_note_for_segment(segment: str) -> tuple:
    """
    Get a random note template for the segment.
    Returns (note_type, note_text)
    """
    templates = NOTE_TEMPLATES.get(segment, NOTE_TEMPLATES["active"])
    return random.choice(templates)


print("Helper functions defined.")
print(f"\nNotes by segment:")
print(f"  Active: ~{determine_notes_count('active')} notes")
print(f"  At-Risk: ~{determine_notes_count('at_risk')} notes")
print(f"  Churned: ~{determine_notes_count('churned')} notes")

Helper functions defined.

Notes by segment:
  Active: ~3 notes
  At-Risk: ~2 notes
  Churned: ~4 notes


In [9]:
def create_customer_notes_sheet(spreadsheet, erp_customers: pd.DataFrame) -> List[Dict]:
    """
    Create customer notes sheet with churn-signal notes.
    
    Returns list of all notes for CSV export.
    """
    print("\n" + "=" * 60)
    print("Creating Customer Notes Sheet")
    print("=" * 60)
    
    all_notes = []
    note_id = 1
    segment_stats = {"active": 0, "at_risk": 0, "churned": 0}
    skipped = 0
    
    try:
        # Get or create customer_notes worksheet
        try:
            ws = spreadsheet.worksheet("customer_notes")
            ws.clear()
            print("  Found existing 'customer_notes' sheet, clearing...")
        except gspread.exceptions.WorksheetNotFound:
            ws = spreadsheet.add_worksheet(title="customer_notes", rows=2000, cols=10)
            print("  Created new 'customer_notes' sheet")
        
        # Prepare headers
        headers = ["note_id", "customer_email", "branch_id", "note_date", "note_type", "note_text", "created_by"]
        rows = [headers]
        
        # Generate notes for each customer
        for idx, customer in erp_customers.iterrows():
            email = customer['email_id']
            segment = customer['website']  # Segment stored in website field
            
            # Skip customers with missing segment (NaN)
            if pd.isna(segment) or segment not in ["active", "at_risk", "churned"]:
                skipped += 1
                continue
            
            # Get note dates for this customer
            note_dates = get_note_dates(segment, START_DATE, END_DATE)
            
            for note_date in note_dates:
                note_type, note_text = get_note_for_segment(segment)
                branch = random.choice(BRANCHES)
                staff = random.choice(STAFF_NAMES)
                
                note = {
                    "note_id": f"N{note_id:05d}",
                    "customer_email": email,
                    "branch_id": branch['branch_id'],
                    "note_date": note_date.strftime("%Y-%m-%d"),
                    "note_type": note_type,
                    "note_text": note_text,
                    "created_by": staff,
                    "segment": segment,  # For tracking
                }
                
                row = [note[col] for col in headers]
                rows.append(row)
                all_notes.append(note)
                segment_stats[segment] += 1
                note_id += 1
            
            if (idx + 1) % 100 == 0:
                print(f"  Processed {idx + 1}/{len(erp_customers)} customers...")
        
        # Write to sheet in batches (Google API limits)
        print(f"\n  Writing {len(rows)} rows to sheet...")
        
        # Write in batches of 500 rows
        batch_size = 500
        for i in range(0, len(rows), batch_size):
            batch = rows[i:i + batch_size]
            start_row = i + 1
            ws.update(range_name=f'A{start_row}', values=batch)
            print(f"    Wrote rows {start_row} to {start_row + len(batch) - 1}")
        
        print(f"\n" + "=" * 60)
        print("INGESTION COMPLETE")
        print("=" * 60)
        print(f"  Total Notes: {len(all_notes)}")
        print(f"  Skipped customers (no segment): {skipped}")
        print(f"\nNotes by segment:")
        for seg, count in segment_stats.items():
            print(f"  {seg}: {count} notes")
        
        return all_notes
        
    except Exception as e:
        print(f"  Error: {e}")
        import traceback
        traceback.print_exc()
        return all_notes

In [10]:
# ============================================================================
# RUN DATA INGESTION
# ============================================================================
#
# This cell creates legacy data in Google Sheets:
# - Branch master data (10 branches)
# - Customer notes (churn signals!)
#
# Runtime: Approximately 2-5 minutes for 500 customers

if spreadsheet and erp_customers is not None:
    print("=" * 60)
    print("Google Sheets Data Ingestion")
    print(f"Target: {spreadsheet.title}")
    print(f"Customers to process: {len(erp_customers)}")
    print("=" * 60)
    
    # Create branches sheet
    create_branches_sheet(spreadsheet)
    
    # Create customer notes sheet
    all_notes = create_customer_notes_sheet(spreadsheet, erp_customers)
else:
    print("Cannot run ingestion:")
    if not spreadsheet:
        print("  - Not connected to Google Sheets")
    if erp_customers is None:
        print("  - ERPNext customers not loaded")
    all_notes = []

Google Sheets Data Ingestion
Target: Apex Bank - Branch Data
Customers to process: 502

Creating Branches Sheet
  Found existing 'branches' sheet, clearing...
  Wrote 10 branches to sheet

Creating Customer Notes Sheet
  Created new 'customer_notes' sheet
  Processed 100/502 customers...
  Processed 200/502 customers...
  Processed 300/502 customers...
  Processed 400/502 customers...
  Processed 500/502 customers...

  Writing 1084 rows to sheet...
    Wrote rows 1 to 500
    Wrote rows 501 to 1000
    Wrote rows 1001 to 1084

INGESTION COMPLETE
  Total Notes: 1083
  Skipped customers (no segment): 2

Notes by segment:
  active: 480 notes
  at_risk: 388 notes
  churned: 215 notes


---
## 4. Data Extraction

Export Google Sheets data to CSV for Databricks ingestion.

In [11]:
def extract_sheet_data(spreadsheet, sheet_name: str) -> List[Dict]:
    """
    Extract all data from a Google Sheet worksheet.
    """
    print(f"\nExtracting '{sheet_name}'...")
    
    try:
        ws = spreadsheet.worksheet(sheet_name)
        records = ws.get_all_records()
        print(f"  Found {len(records)} records")
        return records
    except Exception as e:
        print(f"  Error: {e}")
        return []

In [12]:
def save_to_csv(data: List[Dict], filename: str, output_dir: Path):
    """Save data to CSV file."""
    if not data:
        print(f"  No data for {filename}")
        return
    
    output_dir.mkdir(parents=True, exist_ok=True)
    filepath = output_dir / filename
    
    # Get all unique keys
    all_keys = set()
    for record in data:
        all_keys.update(record.keys())
    fieldnames = sorted(list(all_keys))
    
    with open(filepath, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"  Saved: {filename} ({len(data)} records)")


def save_to_json(data: List[Dict], filename: str, output_dir: Path):
    """Save data to JSON file."""
    if not data:
        print(f"  No data for {filename}")
        return
    
    output_dir.mkdir(parents=True, exist_ok=True)
    filepath = output_dir / filename
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, default=str)
    
    print(f"  Saved: {filename} ({len(data)} records)")

In [13]:
# ============================================================================
# RUN DATA EXTRACTION
# ============================================================================

if spreadsheet:
    print("=" * 60)
    print("Google Sheets Data Extraction")
    print(f"Source: {spreadsheet.title}")
    print(f"Output: {OUTPUT_DIR.absolute()}")
    print("=" * 60)
    
    # Extract data from sheets
    gs_branches = extract_sheet_data(spreadsheet, "branches")
    gs_notes = extract_sheet_data(spreadsheet, "customer_notes")
    
    # Save to CSV
    print("\n" + "=" * 60)
    print("Saving to CSV")
    print("=" * 60)
    
    save_to_csv(gs_branches, "gs_branches.csv", OUTPUT_DIR)
    save_to_csv(gs_notes, "gs_customer_notes.csv", OUTPUT_DIR)
    
    # Save to JSON
    print("\n" + "=" * 60)
    print("Saving to JSON")
    print("=" * 60)
    
    save_to_json(gs_branches, "gs_branches.json", OUTPUT_DIR)
    save_to_json(gs_notes, "gs_customer_notes.json", OUTPUT_DIR)
    
    # Summary
    print("\n" + "=" * 60)
    print("EXTRACTION COMPLETE")
    print("=" * 60)
    print(f"\nFiles saved to: {OUTPUT_DIR.absolute()}")
    print(f"\n  gs_branches.csv       : {len(gs_branches)} records")
    print(f"  gs_customer_notes.csv : {len(gs_notes)} records")
else:
    print("Not connected to Google Sheets - cannot extract data")

Google Sheets Data Extraction
Source: Apex Bank - Branch Data
Output: c:\Users\SulaimanAhmed\Desktop\portfolio\Banking project\banking-churn-databricks\notebooks\exploration\..\..\data\raw

Extracting 'branches'...
  Found 10 records

Extracting 'customer_notes'...
  Found 1083 records

Saving to CSV
  Saved: gs_branches.csv (10 records)
  Saved: gs_customer_notes.csv (1083 records)

Saving to JSON
  Saved: gs_branches.json (10 records)
  Saved: gs_customer_notes.json (1083 records)

EXTRACTION COMPLETE

Files saved to: c:\Users\SulaimanAhmed\Desktop\portfolio\Banking project\banking-churn-databricks\notebooks\exploration\..\..\data\raw

  gs_branches.csv       : 10 records
  gs_customer_notes.csv : 1083 records


---
## Summary

This notebook provides the complete Google Sheets legacy data pipeline:

1. **Configuration** - Auto-load credentials from .env
2. **API Client** - Connect using service account
3. **Ingestion** - Create branches and customer notes
4. **Extraction** - Export to CSV/JSON

### Churn Signal Implementation

| Segment | Notes | Types |
|---------|-------|-------|
| Active | 0-3 | Positive, neutral |
| At-Risk | 2-4 | Warning notes |
| Churned | 2-5 | Negative, closure notes |

### Entity Resolution

Customers are linked via **email**:
```
ERPNext.email_id = Google Sheets.customer_email
```

### Files Generated

```
data/raw/
├── gs_branches.csv         # 10 branches
├── gs_branches.json
├── gs_customer_notes.csv   # Customer notes (churn signal!)
└── gs_customer_notes.json
```

### Next Steps

1. All 4 source systems now complete!
2. Build Bronze layer in dbt (raw ingestion)
3. Build Silver layer (cleaning + unification)
4. Build Gold layer (features + customer_360)

---
*Created for Banking Customer Churn Prediction POC*